#Deskewing pages using OpenCV and Python#
When we think about training OCR, we often focus on refining the software's ability to recognize particular characters, or providing domain-specific vocabulary lists to improve the software's repertoire for recognizing words. But one of the most important things we can do to improve text recognition is to optimize the quality of the images we're asking the software to process in the first place.

The code in this notebook is drawn from a blog post by Leo Ertuna at [Becoming Human](https://becominghuman.ai/how-to-automatically-deskew-straighten-a-text-image-using-opencv-a0c30aed83df). Ertuna has written this deskewing routine as a set of defined functions that would be invoked by invoking a `deskew`  function and passing it the name of an image [i.e., `deskew(image_name)`]. 

This notebook breaks Ertuna's code up into interactive steps to show what's happening along the way, then offers a revision of one of Ertuna's techniques that may offer a slightly improved result.

### A note before we start: this may not be the only problem to solve
The code in this notebook assumes that the problem with the page image is that it's skewed and needs to straightened—this treats the page as a two-dimensional plane. 

That works pretty well if the images are of reasonably flat pages of the kind that we can often get from the sort of imaging labs that many libraries have. But books don't necessarily lay flat, and, depending on the condition of the binding, it may not always possible to flatten the pages for imaging. So the lines of text in some images will appear not just skewed, but actually curved, due to the curvature of the pages. And pages in a book can curl in a number of ways all at once (recall, for instance how much more and how differently the pages in the middle of a thick book curl compared to pages at the beginning or end.)

It's possible to reduce or eliminate the appearance of curvature in the lines of a page image incorporating some of techniques we'll see in this deskewing routine (but adding some others). That's a more complicated problem that we won't take on, but there's a great [blog post by Mark Zucker](https://mzucker.github.io/2016/08/15/page-dewarping.html) that walks through a solution. The blog post offers visualizations of what's happening at each step, so it's an instructive read even if you're not examining the details of the code.

(Zucker's [full code is on GitHub](https://github.com/mzucker/page_dewarp), but it's written for Python 2, rather than Python 3. Robert Sachunsky has a [fork of Zucker's code](https://github.com/bertsky/page_dewarp) for Python 3.) 

## Connect to Google Drive, copy files, and install packages

In [ ]:
#Get access to Google Drive
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cp -r /gdrive/MyDrive/rbs_digital_approaches_2021/data_class/page_images/penn_pr3732_t7_1730b.zip /content/penn_pr3732_t7_1730b.zip
%cd /content/
!unzip penn_pr3732_t7_1730b.zip

In [ ]:
#Install IPyWidgets to provide widgets for experimenting with some variables later
import ipywidgets as widgets
from ipywidgets import interact

#Import necessary Python packages for use in our code. 

#Note that opencv-python is installed by default in Google Colaboratory. If you 
#were working in a different environment, you'd need to be sure it was installed
#using pip

#(The second import is specific to Google Colaboratory and provides a workaround
#to get OpenCV's imshow command to work properly in a Colab notebook.
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

## Opening the image
Run the next cell to get a drop-down list of different images for processing, all with varying levels of skewing. (You only need to run that cell once. Thereafter, you can change the image you're working with by choosing a different image from the select menu.)

In [ ]:
image_select = widgets.Dropdown(
    description='Choose image',\
    options = ['PR3732_T7_1730b_body00' + i for i in ['04.tif', '11.tif', '13.tif', '21.tif', '36.tif', '63.tif', '78.tif', '82.tif']],\
    value = 'PR3732_T7_1730b_body0004.tif',
    style={'description_width': 'initial'})
display(image_select)

In [ ]:
#Identify the skewed image and have OpenCV read it. (This can take a little 
#while, so give it time to complete.)

source_directory = '/content/penn_pr3732_t7_1730b/'
skewed_image = source_directory + image_select.value
im = cv2.imread(skewed_image, cv2.IMREAD_COLOR)
#Let's see what the image looks like: an excellent image, but a little skewed.
cv2_imshow(im)

##Manipulating the image##
Tesseract might well be able to handle an image like this, but recognition of text lines will be better if we can straighten it. Ertuna's script offers a nice example of a workflow for figuring out exactly *how* skewed the image is, then using that measurement to straighten the image. As we proceed, you'll see some of the ways that images that are good for *us* are not as useful for the computer, and vice versa.

In [ ]:
#Make a copy of the image
newImage = im.copy()
#Convery to grayscale
gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
#Apply a Gaussian blur to reduce the effect of any noise in the image
blur = cv2.GaussianBlur(gray, (9, 9), 0)
#Convert the image to inverted black and white (i.e., white text on a 
#black background). Note that Ertuna's script uses Otsu's method for 
#thresholding to black and white.
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2_imshow(thresh)

### Set some variables
Run the next cell to create a few sliders that will allow you to adjust the variables used in the next steps. (You only need to run that cell once. Thereafter, changing the sliders will change the values of the variables used in the subsequent cells.)

In [ ]:
kernel_width = widgets.IntSlider(description = 'Kernel width', \
                                               min=10, max=50, step=5, value=30)
kernel_height = widgets.IntSlider(description='Kernel height', \
                                                 min=1, max=10, step=1, value=5)
num_iterations = widgets.IntSlider(description='Iterations', min=1, \
                      max=10, step=1, value=5)
display(kernel_width) 
display(kernel_height)
display(num_iterations)

###Now things start to get strange...###
Ertuna implements a common approach that may seem counterintuitive at first: we're going to `dilate` the white pixels of the text until they run together to form solid blocks of white. 
The next cell creates a set of sliders so you can play with the values that determine the size of the `kernel` used to dilate those pixels and the number of iterations for dilation (the defaults follow Ertuna's script). 
NOTE: After you've run the next cell to create the sliders, don't run it again—that will just reset the values to their defaults. Instead, make any adjustments to the sliders and then run the cell *below*.

In [ ]:
#The kernel variable defines a shape to use for dilating the pixels: in this 
#case, a rectangle of 30 pixels wide and 5 pixels high. These proportions ensure
# that the text will run together while more or less maintaining the vertical 
#dimensions of the text lines. You could experiment with changing the x and y
#dimensions that are passed to cv2.MORPH_RECT to see how the output changes.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_width.value, kernel_height.value))
#We dilate the pixels using the shape defined by kernel, and perform the operation
#five times. You could try increasing or decreasing the number of iterations to
#see how the output changes.
dilate = cv2.dilate(thresh, kernel, iterations=num_iterations.value)
cv2_imshow(dilate)

This cell finds the boundaries of the dilated white blocks that used to be our text lines and determines their contours. 
I've made one adjustment to Ertuna's script here in using the `RETR_EXTERNAL` method rather than the `RETR_LIST` method that Ertuna used. Ertuna's method retrieves *all* contours that are detected, where `RETR_EXTERNAL` ignores contours that are found *within other contours*. Though I can't say I've tested it entirely systematically, this approach seems to do a better job of detecting text blocks in this eighteenth-century text, where what may be wandering in the baseline of the set type creates some gaps that end up being detected as contours.

In [ ]:
#Determine contours [more info here]
contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#These next steps are not really part of the deskewing sequence. I've included 
#them simply so we can see what's happening. We first create a color version
#of our black-and-white image, then draw a bright green line connecting the
#contour points so we can see the outlines of the detected shapes
show_contours = cv2.cvtColor(dilate, cv2.COLOR_BayerGR2RGB)
draw_contours = cv2.drawContours(show_contours, contours, -1, (115,255,105), 3)
cv2_imshow(draw_contours)

###Finding the rectangle that fits this contour###
Ertuna's script acts on only the largest of the detected areas on the not-unreasonable premise that the skew angle of the largest text block will be a good proxy for the skew angle of the entire page of text. (He notes, though, that other approaches are possible. One might find that the angle of a different block yielded better results, or the average of multiple blocks.)

Having selected the largest contour, the code in the next cell then determines the smallest possible rectangle that could contain the entire contour using `minAreaRect`: basically, we're drawing a straight-sided box around the irregular contour of the text block.



(The code in the following cell is, again, not really part of the deskewing procedure, but I have added it to show what's happening.)

Actually, what `minAreaRect` produces is not exactly a rectangle, but rather some *instructions* for making a rectangle. We get:
- the x, y coordinates of the center point;
- the width and height of the rectangle; and
- the angle of the rectangle (for more on how `minAreaRect` treats this angle, see [this post at *The AI Learner*](https://theailearner.com/tag/cv2-minarearect/).)

To actually draw the rectangle described by `minAreaRect`, the following cell uses OpenCV's `boxPoints` to get the corner points, then draws a series of lines to connect those corners.

In [ ]:
#This sorts the set of contour points in reverse order: the contours of the 
#largest area will be first in this sorted, which will come in handy in the next
#cell.
sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)
#Select the largest detected contour
largest_contour = sorted_contours[0]
#Determine the minimum-area rectangle that would contain that contour
largest_min_area_rect = cv2.minAreaRect(largest_contour)
print(largest_min_area_rect)

In [ ]:
def draw_min_area_rect(cv2minimumarearectangle, base_image) :
  draw_min_area_rect = cv2.cvtColor(base_image, cv2.COLOR_BayerGR2RGB)
  if isinstance(cv2minimumarearectangle, list) == True :
    print(len(cv2minimumarearectangle))
    for rect in cv2minimumarearectangle :
      min_area_box = cv2.boxPoints(rect)
      min_area_box = np.int0(min_area_box)
      draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[0][0], min_area_box[0][1]), \
                                    (min_area_box[1][0], min_area_box[1][1]), (0, 30, 255), 3)
      draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[1][0], min_area_box[1][1]), \
                                    (min_area_box[2][0], min_area_box[2][1]), (0, 30, 255), 3)
      draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[2][0], min_area_box[2][1]), \
                                    (min_area_box[3][0], min_area_box[3][1]), (0, 30, 255), 3)
      draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[3][0], min_area_box[3][1]), \
                                    (min_area_box[0][0], min_area_box[0][1]), (0, 30, 255), 3)
      cv2.putText(draw_min_area_rect, str(rect[-1]), 
                  (int(rect[0][0]) -100, int(rect[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 
                  1, (0, 30, 255, 255), 3)
  else :
    min_area_box = cv2.boxPoints(cv2minimumarearectangle)
    min_area_box = np.int0(min_area_box)
    draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[0][0], min_area_box[0][1]), \
                                  (min_area_box[1][0], min_area_box[1][1]), (0, 30, 255), 3)
    draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[1][0], min_area_box[1][1]), \
                                  (min_area_box[2][0], min_area_box[2][1]), (0, 30, 255), 3)
    draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[2][0], min_area_box[2][1]), \
                                  (min_area_box[3][0], min_area_box[3][1]), (0, 30, 255), 3)
    draw_min_area_rect = cv2.line(draw_min_area_rect, (min_area_box[3][0], min_area_box[3][1]), \
                                  (min_area_box[0][0], min_area_box[0][1]), (0, 30, 255), 3)
    cv2.putText(draw_min_area_rect, str(cv2minimumarearectangle[-1]), 
                (int(cv2minimumarearectangle[0][0]) -100, int(cv2minimumarearectangle[0][1])), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 30, 255, 255), 3)

  return draw_min_area_rect

In [ ]:
largest_min_area = draw_min_area_rect(largest_min_area_rect, dilate)
cv2_imshow(largest_min_area)

###Back to the actual deskewing routine###
Getting the angle of the rectangle's skew is actually easier than drawing the rectangle: we just need the last number produced by `minAreaRect`, but because of the way `minAreaRect` measures the angle, we do need to have a bit of math to make sure the value comes out usable.

In [ ]:
# Determine the angle. Convert it to the value that was originally used to obtain skewed image
largest_rect_angle = largest_min_area_rect[-1]
if largest_rect_angle < -45:
    largest_rect_angle = 90 + largest_rect_angle
print(largest_rect_angle)

###Let's see the deskewed image.###
The code in the next cell takes a few steps to rotate our image 
1. First, we make a copy of our original image
2. Next, we determine the size of the image by getting its height and width (the first two items returned by `shape`)
3. Then, we determine the center of the image by dividing its height and width by two.
4. Next, we construct the rotation we want to happen: rotating the image around its center point by the `angle` we determined in the previous cell, //and keeping the scale of the resulting image the same//??
5. [Need to undertand this part better to explain]

Note how we're using information that we calculated by using what is to us a very strange-looking image, and applying it to our original color image.

In [ ]:
largest_rect_deskew = im.copy()
(h, w) = largest_rect_deskew.shape[:2]
center = (w // 2, h // 2)
# M = cv2.getRotationMatrix2D(center, angle, 1.0)
M = cv2.getRotationMatrix2D(center, largest_rect_angle, 1.0)
deskewed_largest_rect = cv2.warpAffine(largest_rect_deskew, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
cv2_imshow(deskewed_largest_rect)

## A modification of Ertuna's approach for early print?###
Ertuna's approach uses the angle of the largest `minAreaRect` for deskewing. This seems like it would be a pretty sound approach if we were dealing with deskewing, say, a page printed from a laser printer that had subsequently been scanned with the paper skewed slightly on scanner bed. 

While this approach seemed to work well for images from the Penn *Sophonisba* that were significantly skewed, however, it actually seemed to make things slightly *worse* for some pages that were only minimally skewed. I'm not entirely sure what to make of this yet, but my hunch is that this result may arise from the character of pages produced on a printing press. Perhaps if the chase wasn't locked up tightly, different lines on the same page could deviate from horizontal by *different* amounts?

Rather than relying solely on the largest text block, the code below sets a couple of threshold to detect as many text blocks as possible (while rejecting regions that seem like they're probably noise), then figures out an average angle for deskewing the entire page.


In [ ]:
#Going to try averaging angles of multiple minAreaRects...
rects = []
for contour in contours :
  minAreaRect = cv2.minAreaRect(contour)
  #minAreaRects with a height of less than 60 pixels seemed almost always to be
  #artifacts of noise, rather than text blocks we'd actually be interested in.
  if minAreaRect[1][1] > 60 : 
    #Let's ignore any text block that appears to be perfectly horizontal--or 
    #rotated 90 degrees. On a normal page, any text block in the latter category 
    #almost always seems to be noise (like a shadow in the gutter or at the 
    #margin).
    if minAreaRect[-1] not in [-0.0, 0.0, -90.0] :
      rects.append(minAreaRect)
draw_all_rects = draw_min_area_rect(rects, dilate)
cv2_imshow(draw_all_rects)


In [ ]:
#We're only interested in the angles from the minAreaRects, but we also 
#need to keep track of which direction the block is skewed. This creates
#an empty list that will hold a series of tuples: the first element will
#be the angle, and the second will be either 1 or -1)
angle_corrections = []
for rect in rects :
  if rect[-1] < -45 :
    #For angles of less than -45 degrees, construct a tuple with
    #the deviation from 90 degrees as the first term
    #(e.g., 90 - (-1 * 87.6834) = 90 - 87.6834 = 2.3166)
    #as the first term and -1 as the second
    angle_corrections.append((90 - (-1.0 * rect[-1]), -1))
  else :
    #For angles of greater than -45, construct a tuple with
    #the deviation from 90 degrees as the first term
    #(e.g. 90 - 90 + -1.4596 = 90 - 88.5404 = 1.4596)
    #and 1 as the second term
    angle_corrections.append((90 - (90 + rect[-1]), 1))
#Determine the mean of the first terms of all of the tuples
#in our list of angle_corrections
average_angle = np.mean([angle_tuple[0] for angle_tuple in angle_corrections])

#Determine whether the deskew angle should be treated as positive
#or negative by taking the sum of the second terms of all 
#of the tuples (e.g., -1 + -1 + -1 + 1 + -1 = -3)
plus_or_minus = sum(angle_tuple[1] for angle_tuple in angle_corrections)
#If that sum ends up as a positive number, the deskewing angle
#needs to be a negative number
if plus_or_minus > 0 :
  average_angle = -1.0 * average_angle
print(plus_or_minus)
print(average_angle)

In [ ]:
average_angle_deskew = im.copy()
(h, w) = average_angle_deskew.shape[:2]
center = (w // 2, h // 2)
# M = cv2.getRotationMatrix2D(center, angle, 1.0)
M = cv2.getRotationMatrix2D(center, average_angle, 1.0)
deskewed_average_angle = cv2.warpAffine(average_angle_deskew, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
cv2_imshow(deskewed_average_angle)

### Comparing the output
This image overlays the results of the two deskewing approaches in the notebook: the image in red was deskewed using the largest rectangle method, while the one in blue was desweked using the averages of the multiple rectangles. In many cases, the results are *ver* similar, though I think that in some cases the second approach may produce a somewhat better result.

In [ ]:
def tint_image(cv2image, color) :
  if color == 'red' :
    color_value = [0,0,255]
  if color == 'blue' :
    color_value = [255, 0, 0]
  if color == 'green' :
    color_value = [0,255,0]
  tint_gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
  # tint_blur = cv2.GaussianBlur(tint_gray, (9, 9), 0)
  tint_thresh = cv2.threshold(tint_gray, 0, 255, cv2.THRESH_OTSU)[1]
  tint = cv2.cvtColor(tint_thresh, cv2.COLOR_BAYER_GR2BGR)
  tint[np.where((tint==[0,0,0]).all(axis=2))] = color_value
  return tint

tint_red = tint_image(deskewed_largest_rect, 'red')
tint_blue = tint_image(deskewed_average_angle, 'blue')
output = cv2.addWeighted(tint_blue,0.6, tint_red, 0.4, 0)
cv2_imshow(output)


## Clear Google Colab environment

In [ ]:
%cd /content/
!rm -r ./*